<a href="https://colab.research.google.com/github/rst-analytics/rst-analytics2019/blob/master/TrackDechets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Etape 1 : Les données

- Liste des ICPE (fichier CSV), source = http://www.installationsclassees.developpement-durable.gouv.fr
- Liste des ICPE (fichier shape), source = georisque.gouv.fr
- Liste des établissements de la base IREP, année 2017, source = georisque.gouv.fr
- Base Sireine, retraité par C. Quest, version 1er mars 2019, http://data.cquest.org/geo_sirene/last/

# Etape 2 : Petit traitement des données

- Le fichier des établissements IREP est passé dans OO puis ré-exporter en csv pour rendre le csv plus propre (suppression des sauts de lignes intempestives), comme suggéré sur Géorisques.
- Le fichier ICPE en shape est converti en csv (simple export de la base des attributs sous QGIS)

# Etape 3 : On commence !






# **Défi 1 : TackDechet**

Luc Mathis, DRI, en mode freestyle


## Import des libs

In [1]:
import pandas as pd
from pandas.tools.plotting import scatter_matrix
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt


!pip install Unidecode
from unidecode import unidecode


    100% |████████████████████████████████| 245kB 11.9MB/s 


## Import des fichiers (situés sur Google Drive pour être traités via Colab)

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## Chargement des data



### IREP

In [0]:
# Etablissement IREP
etablissement_irep_ini = pd.read_csv("/content/gdrive/My Drive/RST-Analytics/TrackDechets/etablissements_clean.csv",sep=',',
                                    dtype={'Identifiant':str, 'Numero_SIRET':str})

In [94]:
etablissement_irep_df = etablissement_irep_ini.copy()
etablissement_irep_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10047 entries, 0 to 10046
Data columns (total 14 columns):
Identifiant          10047 non-null object
Nom_Etablissement    10047 non-null object
Numero_SIRET         10047 non-null object
Adresse              7367 non-null object
Code_Postal          10047 non-null int64
Commune              10046 non-null object
Departement          10047 non-null object
Region               10047 non-null object
Coordonnees_X        10047 non-null float64
Coordonnees_Y        10047 non-null float64
Code_APE             10047 non-null object
Libelle_APE          10047 non-null object
code_eprtr           2955 non-null object
libelle_eprtr        2955 non-null object
dtypes: float64(2), int64(1), object(11)
memory usage: 1.1+ MB


Base IREP 2017 : 10 000 entrées, avec numéro de SIRET et identifiant GEREP


### ICPE


In [0]:
icpe_ini = pd.read_csv("/content/gdrive/My Drive/RST-Analytics/TrackDechets/installations_classees.csv",sep=',', encoding = 'utf8',
                       dtype={'Identifiant':str})

In [96]:
icpe_df = icpe_ini.copy()
icpe_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52012 entries, 0 to 52011
Data columns (total 10 columns):
Identifiant           52012 non-null object
Nom_etablissement     52012 non-null object
Code postal           52012 non-null int64
Commune               52012 non-null object
Département           52012 non-null object
Régime                52012 non-null object
Statut Seveso         52012 non-null object
Etat d'activité       52012 non-null object
Priorité nationale    52012 non-null object
IED-MTD               52012 non-null object
dtypes: int64(1), object(9)
memory usage: 4.0+ MB


Base ICPE : 52 000 entrées, avec identifiant GEREP ... mais pas les identifiants SIRET

### ICPE, 2ième essai à partir du CSV (export du shapefile)

In [0]:
icpe_shp_ini = pd.read_csv("/content/gdrive/My Drive/RST-Analytics/TrackDechets/icpe-shp-export-attrbuts.csv",sep=',', encoding = 'utf8',
                           dtype={'code_s3ic':str, 'num_siret':str})

In [98]:
icpe_shp_df = icpe_shp_ini.copy()
icpe_shp_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51033 entries, 0 to 51032
Data columns (total 22 columns):
code_s3ic     51033 non-null object
x             51033 non-null int64
y             51033 non-null int64
epsg          51033 non-null int64
nom_ets       51033 non-null object
num_dep       51033 non-null object
cd_insee      51033 non-null object
cd_postal     51033 non-null int64
nomcommune    51033 non-null object
code_naf      20809 non-null object
lib_naf       20809 non-null object
num_siret     40883 non-null object
regime        44400 non-null object
lib_regime    44400 non-null object
ippc          51033 non-null int64
seveso        51032 non-null object
lib_seveso    51032 non-null object
famille_ic    51033 non-null object
url_fiche     51033 non-null object
rayon         23930 non-null float64
precis_loc    51031 non-null float64
lib_precis    51033 non-null object
dtypes: float64(2), int64(5), object(15)
memory usage: 8.6+ MB


Trop cool, on a le code S3IC (Gerep ?) et le code SIRET pour 40883 installations classés !!  Il reste donc environ 10 000 SIRET à trouver.


Bon, il y a un problème, le shapefile a bouffé les caractères UTF ...


In [99]:
icpe_clean_df = pd.merge(icpe_df, icpe_shp_df, left_on='Identifiant', right_on='code_s3ic',how='left')

cols = [10,11,12,13,14,15,16,17,18,22,23,24,25,26,27,28,29,30,31]
icpe_clean_df.drop(icpe_clean_df.columns[cols],axis=1,inplace=True)

icpe_clean_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 52012 entries, 0 to 52011
Data columns (total 13 columns):
Identifiant           52012 non-null object
Nom_etablissement     52012 non-null object
Code postal           52012 non-null int64
Commune               52012 non-null object
Département           52012 non-null object
Régime                52012 non-null object
Statut Seveso         52012 non-null object
Etat d'activité       52012 non-null object
Priorité nationale    52012 non-null object
IED-MTD               52012 non-null object
code_naf              20774 non-null object
lib_naf               20774 non-null object
num_siret             40821 non-null object
dtypes: int64(1), object(12)
memory usage: 5.6+ MB


## 1er essai, croisons IREP et ICPE

Un peu de regex pour normalier les identifiant S3IC dans IREP : 070.00579 ==>  0070.00579


In [0]:
etablissement_irep_df["Identifiant"] =  etablissement_irep_df["Identifiant"].replace(regex=['^'], value='0')
#etablissement_irep_df

In [101]:

result = pd.merge(icpe_clean_df, etablissement_irep_df, left_on='Identifiant', right_on='Identifiant',how='inner')
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9371 entries, 0 to 9370
Data columns (total 26 columns):
Identifiant           9371 non-null object
Nom_etablissement     9371 non-null object
Code postal           9371 non-null int64
Commune_x             9371 non-null object
Département           9371 non-null object
Régime                9371 non-null object
Statut Seveso         9371 non-null object
Etat d'activité       9371 non-null object
Priorité nationale    9371 non-null object
IED-MTD               9371 non-null object
code_naf              6056 non-null object
lib_naf               6056 non-null object
num_siret             8297 non-null object
Nom_Etablissement     9371 non-null object
Numero_SIRET          9371 non-null object
Adresse               6842 non-null object
Code_Postal           9371 non-null int64
Commune_y             9370 non-null object
Departement           9371 non-null object
Region                9371 non-null object
Coordonnees_X         9371 non-null

Sur les 10000 IREP, 9371 sont dans la base ICPE. Parmi ces 9371 entrées communes, la base ICPE n'apporte que 8300 numéro SIRET alors que le fichier IREP en donne 9371. On gagne donc environ 1000 SIRET en croisant les bases. Plus que 9000 à trouver.  Sans parler des IREP qui ne sont pas ICPE ...

---



### Mettons la base au propre

In [102]:
icpe_x_irep = pd.merge(icpe_clean_df, etablissement_irep_df, left_on='Identifiant', right_on='Identifiant',how='outer')

icpe_x_irep.columns = ['icpe_code_s3ic', 'icpe_nom_ets','icpe_cd_postal','icpe_nomcommune',
                       'icpe_dep','icpe_regime','icpe_statut_seveso', 'icpe_etat_activite',
                       'icpe_priorite_nationale','icpe_IED-MTD',
                       'icpe_code_naf','icpe_lib_naf','icpe_num_siret',
                       'irep_Nom_Etablissement','irep_Numero_SIRET','irep_Adresse','irep_Code_Postal',
                       'irep_Commune','irep_Departement','irep_Region','irep_Coordonnees_X','irep_Coordonnees_Y',
                       'irep_Code_APE','irep_Libelle_APE','irep_code_eprtr','irep_libelle_eprtr']
icpe_x_irep.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52688 entries, 0 to 52687
Data columns (total 26 columns):
icpe_code_s3ic             52688 non-null object
icpe_nom_ets               52012 non-null object
icpe_cd_postal             52012 non-null float64
icpe_nomcommune            52012 non-null object
icpe_dep                   52012 non-null object
icpe_regime                52012 non-null object
icpe_statut_seveso         52012 non-null object
icpe_etat_activite         52012 non-null object
icpe_priorite_nationale    52012 non-null object
icpe_IED-MTD               52012 non-null object
icpe_code_naf              20774 non-null object
icpe_lib_naf               20774 non-null object
icpe_num_siret             40821 non-null object
irep_Nom_Etablissement     10047 non-null object
irep_Numero_SIRET          10047 non-null object
irep_Adresse               7367 non-null object
irep_Code_Postal           10047 non-null float64
irep_Commune               10046 non-null object
irep_Dep

In [103]:
filter =  (icpe_x_irep['icpe_num_siret'].isna() & icpe_x_irep['irep_Numero_SIRET'].isna())
icpe_w = icpe_x_irep[filter]
len(icpe_w)

10117

## Sireine, nous voilà

Le fichier Sireine est trop gros : 8Go, 

On préfiltre sous bash en éliminant : 
* Les professions libérales (NATETAB =  5)
* Les entrepreneurs individuels (NATETAB =  9)
* Les copro (NJ = 9110), les associations (NJ = 9220), les communes (7210), les SCI  (6540), les CE (8311), les services déconcentrés (7172), les services départementaux (7220), les services sociaux communaux (7361), les associations foncières (7323), les EPA (7389)
* Les APE Justice (8423Z), Enseignement secondaire général (8531Z), Administration publique générale (8411Z) ,Enseignement primaire ( 8520Z)

Puis on ne garde que quelques colonnes !

```

csvgrep -i -c 41 -r "5|9" geo_sirene.csv > temp.csv

csvgrep -i -c 71 -r "(9110)|(9220)|(7210)|(6540)|(8311)|(7172)|(7220)|(7361)|(7323)|(7389)" temp.csv > temp2.csv

csvgrep -i -c 43 -r "(8423Z)|(8531Z)|(8411Z)|(8520Z)" temp2.csv > temp3.csv

csvcut -c 1,2,3,4,10,11,21,23,29,37,43,44,47,61,63,64,65,70,71,72,73,101,102,105 temp3.csv > sireine_light.csv

```

Préparation des données ICPE : tout mettre en minuscule + supprimer les parenthèses
Apparier sur le CP + le nom par rapport à Sireine ...








In [13]:
import sys
sys.getdefaultencoding()

'utf-8'

In [0]:
icpe_x_irep["icpe_nom_ets"] =  icpe_x_irep["icpe_nom_ets"].str.replace(u"é", "e")
icpe_x_irep["icpe_nom_ets"] =  icpe_x_irep["icpe_nom_ets"].str.replace(u"è", "e")

icpe_x_irep["icpe_nom_ets"] =  icpe_x_irep["icpe_nom_ets"].str.lower()
icpe_x_irep["icpe_nomcommune"] =  icpe_x_irep["icpe_nomcommune"].str.lower()

#On supprime les (....) ; exemple MON_ENTREPRISE (ex TOn_ENTREPRISE)
icpe_x_irep["icpe_nom_ets"] =  icpe_x_irep["icpe_nom_ets"].replace(regex=['\(.*\)'], value='')
icpe_x_irep["icpe_nom_ets"] =  icpe_x_irep["icpe_nom_ets"].replace(regex=['sas'], value='')
icpe_x_irep["icpe_nom_ets"] =  icpe_x_irep["icpe_nom_ets"].replace(regex=['sarl'], value='')
icpe_x_irep["icpe_nom_ets"] =  icpe_x_irep["icpe_nom_ets"].replace(regex=['sa'], value='')
icpe_x_irep["icpe_nom_ets"] =  icpe_x_irep["icpe_nom_ets"].replace(regex=['gaec'], value='')
icpe_x_irep["icpe_nom_ets"] =  icpe_x_irep["icpe_nom_ets"].replace(regex=['earl'], value='')
#icpe_x_irep["icpe_nom_ets"] =  icpe_x_irep["icpe_nom_ets"].replace(regex=['ex .*'], value='')

## L'import de la base Sireine préparée (enfin)

In [105]:
sireine_ini = pd.read_csv("/content/gdrive/My Drive/RST-Analytics/TrackDechets/sireine_light-2.csv",sep=',',
                         dtype={'SIREN':str, 'NIC':str})

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [106]:
sireine_df = sireine_ini.copy()
sireine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6469981 entries, 0 to 6469980
Data columns (total 22 columns):
SIREN            object
NIC              object
L1_NORMALISEE    object
L2_NORMALISEE    object
L1_DECLAREE      object
L2_DECLAREE      object
CODPOS           float64
RPET             int64
LIBCOM           object
ENSEIGNE         object
APET700          object
LIBAPET          object
LIBTEFET         object
NOMEN_LONG       object
NOM              object
PRENOM           object
CIVILITE         float64
ADR_MAIL         float64
NJ               float64
LIBNJ            object
APEN700          object
geo_adresse      object
dtypes: float64(4), int64(1), object(17)
memory usage: 1.1+ GB


On met les noms en minuscules !

In [0]:
sireine_df["NOMEN_LONG"] =  sireine_df["NOMEN_LONG"].str.lower()
sireine_df["L1_NORMALISEE"] =  sireine_df["L1_NORMALISEE"].str.lower()
sireine_df["L1_DECLAREE"] =  sireine_df["L1_DECLAREE"].str.lower()
sireine_df["LIBCOM"] =  sireine_df["LIBCOM"].str.lower()


sireine_df["L1_NORMALISEE"] =  sireine_df["L1_NORMALISEE"].str.replace(u"é", "e")
sireine_df["L1_NORMALISEE"] =  sireine_df["L1_NORMALISEE"].str.replace(u"è", "e")


sireine_df["L1_NORMALISEE"] =  sireine_df["L1_NORMALISEE"].replace(regex=['sas'], value='')
sireine_df["L1_NORMALISEE"] =  sireine_df["L1_NORMALISEE"].replace(regex=['sarl'], value='')
sireine_df["L1_NORMALISEE"] =  sireine_df["L1_NORMALISEE"].replace(regex=['sa'], value='')
sireine_df["L1_NORMALISEE"] =  sireine_df["L1_NORMALISEE"].replace(regex=['gaec'], value='')
sireine_df["L1_NORMALISEE"] =  sireine_df["L1_NORMALISEE"].replace(regex=['earl'], value='')


On tente un appariement CP + Nom

In [0]:
icpe_x_sireine = pd.merge(icpe_x_irep, sireine_df, 
                          left_on=['icpe_cd_postal', 'icpe_nom_ets'],
                          right_on=['CODPOS', 'L1_NORMALISEE'],
                          how='left')

In [114]:
icpe_x_sireine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55478 entries, 0 to 55477
Data columns (total 48 columns):
icpe_code_s3ic             55478 non-null object
icpe_nom_ets               54802 non-null object
icpe_cd_postal             54802 non-null float64
icpe_nomcommune            54802 non-null object
icpe_dep                   54802 non-null object
icpe_regime                54802 non-null object
icpe_statut_seveso         54802 non-null object
icpe_etat_activite         54802 non-null object
icpe_priorite_nationale    54802 non-null object
icpe_IED-MTD               54802 non-null object
icpe_code_naf              22099 non-null object
icpe_lib_naf               22099 non-null object
icpe_num_siret             42837 non-null object
irep_Nom_Etablissement     10656 non-null object
irep_Numero_SIRET          10656 non-null object
irep_Adresse               7824 non-null object
irep_Code_Postal           10656 non-null float64
irep_Commune               10655 non-null object
irep_Dep

Une apparieent brut entre les bases (sur le nom et le CP), permet d'avoir un recouvrement d'à peine 11 000 entrées. Reste à savoir si cet appariement donne de l'information supplémentaire : les SIREN obtenus étaient-ils connus ?

In [115]:
filter = (icpe_x_sireine['icpe_num_siret'].notna() | icpe_x_sireine['irep_Numero_SIRET'].notna() | icpe_x_sireine['SIREN'].notna())
result = icpe_x_sireine[filter]
len(result)

46514

Un peu mieux mais ce n'est pas génial, il va falloir faire mieux !!! On ne gagner "que" 3000 SIRET avec l'appariement bourin !
 



In [116]:
filter =  (icpe_x_sireine['icpe_num_siret'].isna() & icpe_x_sireine['irep_Numero_SIRET'].isna() & icpe_x_sireine['SIREN'].isna())
icpe_w = icpe_x_sireine[filter]
len(icpe_w)

8964

In [0]:
icpe_w

### Mise au propre du CSV + Export

In [117]:
icpe_x_sireine_df = icpe_x_sireine.copy()

icpe_siret_df = pd.concat([icpe_x_sireine_df["icpe_code_s3ic"],icpe_x_sireine_df["icpe_num_siret"],icpe_x_sireine_df["irep_Numero_SIRET"],
                           icpe_x_sireine["SIREN"],icpe_x_sireine["NIC"]],axis=1)

icpe_siret_df.info()
icpe_siret_df.to_csv("/content/gdrive/My Drive/RST-Analytics/TrackDechets/icpe_siret.csv")



<class 'pandas.core.frame.DataFrame'>
Int64Index: 55478 entries, 0 to 55477
Data columns (total 5 columns):
icpe_code_s3ic       55478 non-null object
icpe_num_siret       42837 non-null object
irep_Numero_SIRET    10656 non-null object
SIREN                16265 non-null object
NIC                  16265 non-null object
dtypes: object(5)
memory usage: 2.5+ MB


In [121]:

icpe_x_sireine.info()
icpe_x_sireine.to_csv("/content/gdrive/My Drive/RST-Analytics/TrackDechets/icpe_siret_full.csv")


<class 'pandas.core.frame.DataFrame'>
Int64Index: 55478 entries, 0 to 55477
Data columns (total 48 columns):
icpe_code_s3ic             55478 non-null object
icpe_nom_ets               54802 non-null object
icpe_cd_postal             54802 non-null float64
icpe_nomcommune            54802 non-null object
icpe_dep                   54802 non-null object
icpe_regime                54802 non-null object
icpe_statut_seveso         54802 non-null object
icpe_etat_activite         54802 non-null object
icpe_priorite_nationale    54802 non-null object
icpe_IED-MTD               54802 non-null object
icpe_code_naf              22099 non-null object
icpe_lib_naf               22099 non-null object
icpe_num_siret             42837 non-null object
irep_Nom_Etablissement     10656 non-null object
irep_Numero_SIRET          10656 non-null object
irep_Adresse               7824 non-null object
irep_Code_Postal           10656 non-null float64
irep_Commune               10655 non-null object
irep_Dep

## Appariement plus "malin"

- On isole les ICPE sans SIRET à ce stade
- Pour chaque ICPE, on cherche les entreprises dans SIREN avec le même code postal
- On calcule la distance de levenstein entre le nom de l'ICPE et chaque entreprise. S'il y en a une dont la distance est faible, on le retient en lui affectant un score de confiance

In [0]:
#TODO


In [0]:
#for row in icpe_w.iterrows():
#    print icpe_w["icpe_code_s3ic"]

for i in range(2,3):
  row = icpe_w.iloc[i]
  filter = (sireine_df["CODPOS"]==row["icpe_cd_postal"])
  result = sireine_df[filter]
  print(row["icpe_nom_ets"])
  print(row["icpe_cd_postal"])
  print(result['CODPOS'])
  
            
                     
  
  





In [0]:
result.head(5000)